# Processing

This notebook reads in the Maryland State Bureau of Elections reports from January-September 2018, September 2014 and September 2016, which were converted from PDF to CSV using [Tabula](https://tabula.technology/), an open-source tool "for liberating data tables trapped inside PDF files." The files for each month and year are processed and combined into cleaned data files for analysis (see `02_analysis.ipynb`).

The following CSV files are in the `input/` folder for each month and year:
- `*_totals.csv`: total active registration, by county and party
- `*_changes.csv`: voter registration changes, by county and change type (address or name, or changes from* a particular party)
- `*_new.csv`: new registrations, by party and method of registration
- `*_removals.csv`: removals from the registered voter list, by party and reason for removal

The cleaned files are saved in the `output/` folder.

In [1]:
# Added by Steve
read.csv('input/01_2018_totals.csv')

county,DEM,REP,GRN,LIB,UNAF,OTH,TOTAL,date,CONF.MAILING,INACTIVE
ALLEGANY,"13,864","21,284",135,284,"7,117",302,"42,986",1/31/18,234,"2,981"
ANNE ARUNDEL,"159,780","134,468",832,"2,708","82,760","1,223","381,771",1/31/18,"4,660","21,399"
BALTIMORE CITY,"301,200","31,039","1,264","1,416","45,782","1,658","382,359",1/31/18,"5,347","27,109"
BALTIMORE CO.,"305,849","142,241","1,449","3,096","90,743","4,448","547,826",1/31/18,"6,215","28,367"
CALVERT,"23,607","26,554",136,489,"13,265",393,"64,444",1/31/18,"2,012","4,291"
CAROLINE,"6,853","9,156",29,107,"3,821",117,"20,083",1/31/18,98,"1,039"
CARROLL,"32,170","63,176",264,899,"24,039",823,"121,371",1/31/18,663,"7,134"
CECIL,"21,232","27,543",152,423,"13,992",561,"63,903",1/31/18,"1,447","7,520"
CHARLES,"66,193","25,331",155,479,"18,001",491,"110,650",1/31/18,628,"7,683"
DORCHESTER,"9,947","8,080",26,116,"2,981",98,"21,248",1/31/18,131,"1,366"


In [1]:
## functions to process `totals`, `changes`, `new` and `removals` files

suppressMessages(library('tidyverse'))
suppressMessages(library('lubridate'))
suppressMessages(library('janitor'))
suppressMessages(library('data.table'))

process_totals <- function (month_yr) {
  df <- read.csv(paste0('input/', month_yr, '_totals.csv'), stringsAsFactors = F)
  df <- df %>% mutate(date = mdy(date),
             DEM = as.numeric(str_replace(DEM,pattern = '\\,', 
                                          replacement = '')),
             REP = as.numeric(str_replace(REP,pattern = '\\,', 
                                          replacement = '')),
             GRN = as.numeric(str_replace(GRN,pattern = '\\,', 
                                          replacement = '')),
             LIB = as.numeric(str_replace(LIB,pattern = '\\,', 
                                          replacement = '')),
             UNAF = as.numeric(str_replace(UNAF,pattern = '\\,', 
                                           replacement = '')),
             OTH = as.numeric(str_replace(OTH,pattern = '\\,', 
                                          replacement = '')),
             TOTAL = as.numeric(str_replace(TOTAL,pattern = '\\,', 
                                            replacement = '')),
             CONF.MAILING = as.numeric(str_replace(CONF.MAILING,pattern = '\\,', 
                                                   replacement = '')),
             INACTIVE = as.numeric(str_replace(INACTIVE,pattern = '\\,', 
                                               replacement = ''))) %>% adorn_totals('row')
  
  colnames(df) <- tolower(colnames(df))
  
  df <- df %>%  mutate(date = ifelse(date == '-', lag(date), date), 
           dem_perc = dem/total * 100, 
           rep_perc = rep/total * 100,
           grn_perc = grn/total * 100,
           lib_perc = lib/total * 100, 
           unaf_perc = unaf/total * 100,
           oth_perc = oth/total * 100)
  return(df)
}

process_changes <- function (month_yr) {
  df <- read.csv(paste0('input/', month_yr, '_changes.csv'), stringsAsFactors = F)
  df <- df %>% mutate(date = mdy(date),
                      ADDRESS = as.numeric(str_replace(ADDRESS,pattern = '\\,', 
                                                       replacement = '')),
                      NAME = as.numeric(str_replace(NAME,pattern = '\\,', 
                                                    replacement = '')),
                      DEM = as.numeric(str_replace(DEM,pattern = '\\,', 
                                                   replacement = '')),
                      REP = as.numeric(str_replace(REP,pattern = '\\,', 
                                                   replacement = '')),
                      GRN = as.numeric(str_replace(GRN,pattern = '\\,', 
                                                   replacement = '')),
                      LIB = as.numeric(str_replace(LIB,pattern = '\\,', 
                                                   replacement = '')),
                      UNAF = as.numeric(str_replace(UNAF,pattern = '\\,', 
                                                    replacement = '')),
                      OTH = as.numeric(str_replace(OTH,pattern = '\\,', 
                                                   replacement = '')),
                      TOTAL = as.numeric(str_replace(TOTAL,pattern = '\\,', 
                                                     replacement = ''))) %>% adorn_totals('row') 
  colnames(df) <- tolower(colnames(df))
  
  df <- df %>% 
  
    mutate(date = ifelse(date == '-', lag(date), date), 
           dem_perc = dem/total * 100, 
           rep_perc = rep/total * 100,
           grn_perc = grn/total * 100,
           lib_perc = lib/total * 100, 
           unaf_perc = unaf/total * 100,
           oth_perc = oth/total * 100)
  return(df)
}

process_new <- function (month_yr) {
  df <- read.csv(paste0('input/', month_yr, '_new.csv'), stringsAsFactors = F)
  df <- df %>% mutate(date = mdy(date),
           DEM = as.numeric(str_replace(DEM,pattern = '\\,', 
                                        replacement = '')),
           REP = as.numeric(str_replace(REP,pattern = '\\,', 
                                        replacement = '')),
           GRN = as.numeric(str_replace(GRN,pattern = '\\,', 
                                        replacement = '')),
           LIB = as.numeric(str_replace(LIB,pattern = '\\,', 
                                        replacement = '')),
           UNAF = as.numeric(str_replace(UNAF,pattern = '\\,', 
                                         replacement = '')),
           OTH = as.numeric(str_replace(OTH,pattern = '\\,', 
                                        replacement = '')),
           TOTAL = as.numeric(str_replace(TOTAL,pattern = '\\,', 
                                          replacement = '')),
           DUPS = as.numeric(str_replace(DUPS,pattern = '\\,', 
                                         replacement = ''))) %>% adorn_totals('row') 
  
  colnames(df) <- tolower(colnames(df))
  
  df <- df %>% 
    mutate(date = ifelse(date == '-', lag(date), date), 
           dem_perc = dem/total * 100, 
           rep_perc = rep/total * 100,
           grn_perc = grn/total * 100,
           lib_perc = lib/total * 100, 
           unaf_perc = unaf/total * 100,
           oth_perc = oth/total * 100)
  return(df)
}

process_removals <- function (month_yr) {
  df <- read.csv(paste0('input/', month_yr, '_removals.csv'), stringsAsFactors = F)
  df <- df %>% mutate(date = mdy(date),
                      DEM = as.numeric(str_replace(DEM,pattern = '\\,', 
                                                   replacement = '')),
                      REP = as.numeric(str_replace(REP,pattern = '\\,', 
                                                   replacement = '')),
                      GRN = as.numeric(str_replace(GRN,pattern = '\\,', 
                                                   replacement = '')),
                      LIB = as.numeric(str_replace(LIB,pattern = '\\,', 
                                                   replacement = '')),
                      UNAF = as.numeric(str_replace(UNAF,pattern = '\\,', 
                                                    replacement = '')),
                      OTH = as.numeric(str_replace(OTH,pattern = '\\,', 
                                                   replacement = '')),
                      TOTAL = as.numeric(str_replace(TOTAL,pattern = '\\,', 
                                                     replacement = ''))) %>% adorn_totals('row')
  colnames(df) <- tolower(colnames(df))
  
  df <- df %>% 
    mutate(date = ifelse(date == '-', lag(date), date), 
           dem_perc = dem/total * 100, 
           rep_perc = rep/total * 100,
           grn_perc = grn/total * 100,
           lib_perc = lib/total * 100, 
           unaf_perc = unaf/total * 100,
           oth_perc = oth/total * 100)
  return(df)
}

In [25]:
## apply functions to files

totals.List <- list()
changes.List <- list()
new.List <- list()
removals.List <- list()

for (i in c('01_2018', '02_2018', '03_2018', '04_2018', '05_2018', '06_2018', '07_2018', 
            '08_2018', '09_2018', '09_2016', '09_2014')) {
    totals.List[[i]] <- process_totals(i)
    changes.List[[i]] <- process_changes(i)
    new.List[[i]] <- process_new(i)
    removals.List[[i]] <- process_removals(i)
    }

totals <- rbindlist(totals.List)
changes <- rbindlist(changes.List)
new <- rbindlist(new.List)
removals <- rbindlist(removals.List)

totals$date <- ymd(totals$date)
changes$date <- ymd(changes$date)
new$date <- ymd(new$date)
removals$date <- ymd(removals$date)

In [27]:
## write to csv in `output/` folder

write_csv(totals, 'output/totals.csv')
write_csv(changes, 'output/changes.csv')
write_csv(new, 'output/new.csv')
write_csv(removals, 'output/removals.csv')